In [ ]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "./data/202512_202512_연령별인구현황_월간.csv",
    encoding="cp949"
)

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype("Int64")

# 3. 시/군 단위만 추출
# - 시도: code % 100_000_000 == 0
# - 시군구: code % 100_000_000 != 0 AND code % 100_000 == 0
sigungu = df[
    (df["행정코드"] % 100_000_000 != 0) &
    (df["행정코드"] % 100_000 == 0)
].copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2025년12월_계_총인구수",
    "2025년12월_계_20~24세",
    "2025년12월_계_25~29세",
    "2025년12월_계_30~34세",
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# 5. 청년(20~34) 인구 계산
sigungu["청년(20~34)인구"] = (
    sigungu["2025년12월_계_20~24세"]
    + sigungu["2025년12월_계_25~29세"]
    + sigungu["2025년12월_계_30~34세"]
)

sigungu["청년비율(20~34)"] = (
    sigungu["청년(20~34)인구"]
    / sigungu["2025년12월_계_총인구수"]
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    "서울특별시", "경기도", 
    "인천광역시", "광주광역시", "울산광역시",
    "부산광역시", "대구광역시", "대전광역시"
]

sigungu["상위지역"] = sigungu["지역명"].str.split().str[0]

local_sigungu = sigungu[
    ~sigungu["상위지역"].isin(exclude_regions) & (sigungu["지역명"].str.split().str.len() == 2)
].copy()

# 7. 결과 테이블 구성
result = local_sigungu[
    ["지역명", "2025년12월_계_총인구수", "청년(20~34)인구", "청년비율(20~34)"]
].rename(columns={"2025년12월_계_총인구수": "총인구"})

result["청년비율(20~34)"] = (result["청년비율(20~34)"] * 100).round(2)

# 8. 결과 출력
print("=== [지방 시/군] 청년비율(20~34) 높은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=False).head(50))

print("=== [지방 시/군] 청년비율(20~34) 낮은 지역 TOP 20 ===")
display(result.sort_values("청년비율(20~34)", ascending=True).head(50))

print("=== [지방 시/군] 청년인구(20~34) 많은 지역 TOP 20 ===")
display(result.sort_values("청년(20~34)인구", ascending=False).head(50))



In [23]:
# 데이터 불러오기
import pandas as pd
import re 

pd.set_option('display.max_rows', 10000)

df = pd.read_csv(
    "../../data/202601_202601_주민등록인구및세대현황_월간.csv",
    encoding="cp949"
)

# 6. 제외할 광역 단위 정의
exclude_regions = [
    "4700000000",   # 경상북도
    "4800000000",   # 경상남도
    "4600000000",   # 전라남도
    "5200000000",   # 전북특별자치도
    "4300000000",   # 충청북도
    "4400000000",   # 충청남도
    "5100000000",   # 강원특별자치도
    "5000000000"    # 제주특별자치도
]

exclude_regions2 = [
    "41",   # 서울특별시 
    "11",   # 경기도
    "26",   # 부산광역시
    "28",   # 인천광역시
    "27",   # 대구광역시 
    "30",   # 대전광역시
    "29",   # 광주광역시
    "31",   # 울산광역시
]

df["지역명"] = df["행정구역"].str.split("(").str[0].str.strip()
df["행정코드"] = df["행정구역"].str.extract(r"\((\d+)\)").astype(str)

sigungu = df.copy()

# 4. 숫자 컬럼 정리
num_cols = [
    "2026년01월_총인구수"
]

for c in num_cols:
    sigungu[c] = (
        sigungu[c]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(int)
    )

# 1) exclude_regions에 정확히 포함되는 코드 제외
m1 = sigungu["행정코드"].isin(exclude_regions)

# 2) exclude_regions2로 시작하는 코드 중에서,
#    prefix 뒤가 전부 0인 형태(prefix + 0+)는 "포함" 예외
prefix_pat = r"^(?:" + "|".join(map(re.escape, exclude_regions2)) + r")"
m2_starts = sigungu["행정코드"].str.match(prefix_pat)

m2_all_zero_after = sigungu["행정코드"].str.match(prefix_pat + r"0+$")
m2_exclude = m2_starts & ~m2_all_zero_after   # 시작은 하는데, 뒤가 전부 0은 아니면 제외

# 최종: 제외 대상(m1 또는 m2_exclude)을 빼고 남김
sigungu = sigungu[~(m1 | m2_exclude)].copy()

# 7. 결과 테이블 구성
result = sigungu[
    ["지역명", "2026년01월_총인구수"]
].rename(columns={"2026년01월_총인구수": "총인구"})


display(result.sort_values("총인구", ascending=False).head(10))

,지역명,총인구
87,경기도,13736642
0,서울특별시,9299701
26,부산광역시,3239711
53,인천광역시,3053308
43,대구광역시,2352249
73,대전광역시,1440550
67,광주광역시,1390658
79,울산광역시,1091281
263,경상남도 창원시,990407
164,충청북도 청주시,856629
